In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import io
import csv

Found existing installation: tensorflow-hub 0.16.1
Uninstalling tensorflow-hub-0.16.1:
  Would remove:
    /Users/esling/Library/Python/3.9/lib/python/site-packages/tensorflow_hub-0.16.1.dist-info/*
    /Users/esling/Library/Python/3.9/lib/python/site-packages/tensorflow_hub/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [ ]:
model_yam = hub.load('https://tfhub.dev/google/yamnet/1')

In [ ]:
print(model_yam)

In [ ]:
window_size = 8000
n_windows = 6
tst_silence = np.zeros(n_windows * window_size)
scores, embeddings, mel_spec = model_yam(tst_silence)

In [ ]:
print(scores.shape)
print(embeddings.shape)

In [ ]:
# Find the name of the class with the top score when mean-aggregated across frames.
def class_names_from_csv(class_map_csv_text):
  """Returns list of class names corresponding to score vector."""
  class_map_csv = io.StringIO(class_map_csv_text)
  class_names = [display_name for (class_index, mid, display_name) in csv.reader(class_map_csv)]
  class_names = class_names[1:]  # Skip CSV header
  return class_names
class_map_path = model_yam.class_map_path().numpy()
class_names = class_names_from_csv(tf.io.read_file(class_map_path).numpy().decode('utf-8'))
class_names[np.argmax(scores)]

In [ ]:
print(class_map_path)

In [30]:
import IPython
IPython.display.Audio("01_cat.wav")

In [ ]:
import scipy
import librosa as li
import soundfile as sf
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv
import io
import os

# Keys of interest
keys = ["Cat", "Meow", "Purr", "Animal"]
num_samples = {k:0 for k in keys}
# Original and target folders
original_dir = "cats/"
target_dir = "cats_clean/"

# Signal properties
m_sr = 16000
win_size = int(0.96 * m_sr)
hop_size = int(0.48 * m_sr)
# Load the model.
model = hub.load('https://tfhub.dev/google/yamnet/1')

# Find the name of the class with the top score when mean-aggregated across frames.
def class_names_from_csv(class_map_csv_text):
    """Returns list of class names corresponding to score vector."""
    class_map_csv = io.StringIO(class_map_csv_text)
    class_names = [display_name for (class_index, mid, display_name) in csv.reader(class_map_csv)]
    class_names = class_names[1:]  # Skip CSV header
    return class_names
class_map_path = model.class_map_path().numpy()
class_names = class_names_from_csv(tf.io.read_file(class_map_path).numpy().decode('utf-8'))

# Parse the set of sounds
for file in os.listdir(original_dir):
    print(f'- {file}')
    waveform, sr = li.load(original_dir + file)
    waveform = li.resample(waveform, sr, 16000)
    print('    . Modeling')
    scores, embeddings, log_mel_speactrogram = model(waveform)
    print('    . Parsing windows')
    for w in range(scores.shape[0]):
        cur_w = waveform[w*hop_size:(w*hop_size + win_size)]
        cur_class = class_names[scores[w].numpy().argmax()]
        cur_score = scores[w].numpy().max()
        if (cur_class in keys) and cur_score > 0.9:
            sf.write(f"{target_dir}/{cur_class}_{num_samples[cur_class]}.wav", cur_w, 16000)
            num_samples[cur_class] += 1
print(num_samples)